In [1]:
import numpy as np
import pandas as pd
import pickle 
from collections import defaultdict
import sys
import os
import nltk
from nltk.corpus import words
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
from HATN import *

Using TensorFlow backend.


In [2]:
data_file = os.path.join(os.getcwd(), '../Data/processed.csv')
data = pd.read_csv(open(data_file, 'r'))

In [3]:
# data.loc[0]
data['decoded'] = data['decoded'].astype('str') 

In [ ]:
import re
def split_email(email):
    ls = re.split("\n|\. |<.*?>|\t|.*?=.*?|.*?: .*?", email)
    return ls

processed = data['decoded'].apply(split_email)

In [ ]:
sender = [item[0] for item in processed]

In [ ]:
sender = [item.split() for item in sender]
sender = [item[1] for item in sender if len(item) > 1]

In [ ]:
subjects = data['decoded'].apply(lambda x : re.findall('Subject: .*', x))

In [ ]:
#import itertools
#subject = list(itertools.chain.from_iterable(subjects))
type(subjects[1])

In [ ]:
def getSubject (item):
    if len(item) > 0:
        return item[0][9:]
    else:
        return ' '
subject = [getSubject(item) for item in subjects]

In [ ]:
data['email_subjects'] = subject

In [ ]:
for idx, i in enumerate(subjects):
    if len(i) > 1:
        print(idx)

In [4]:
stop_words = set(stopwords.words('english')) 
eng_words = set(words.words())
ps = PorterStemmer()
def filterEnglish (word):
    if len(word) > 40:
        return False
    elif not word.lower() in eng_words or word.lower() in stop_words:
        return False
    return True

#lst = str.split(data['decoded'].loc[1])
#lst = nltk.word_tokenize(data['decoded'].loc[1])
#lstfilter = [word for word in lst if len(word) < 40]
#result = [word for word in lst if filterEnglish(word)]

In [5]:
# contents = [nltk.word_tokenize(email) for email in data['decoded']]
contents = data['decoded'].map(nltk.word_tokenize)

In [6]:
def filterContent (lst):
    return list(filter(filterEnglish, lst))
# filtered = [filterContent(item) for item in contents[:10000]]
filtered = contents.map(filterContent)

In [7]:
filter_ = list(map(lambda x: " ".join(x), filtered))

In [8]:
len(filter_)

75422

In [ ]:
with open(os.path.join(os.getcwd(), '../Data/processed.csv'), 'w') as f:
    data.to_csv(f)

In [ ]:
import csv
with open(os.path.join(os.getcwd(), '../Data/EmailContent.csv'), 'w+') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(filter_)

In [20]:
with open(os.path.join(os.getcwd(), '../Data/EmailContentwithLabels.txt'), 'w+') as f:
    for idx, line in enumerate(filter_):
        f.write(str(int(data['label'][idx])) + ',' + line + '<SEP>')

ValueError: cannot convert float NaN to integer

In [15]:
with open(os.path.join(os.getcwd(), '../Data/EmailContentwithLabels.txt'), 'r+') as f:
    d = f.read()
    
ls = d.split('<SEP>')
ls = [i.split(',') for i in ls]

In [19]:
labels = [int(i[0]) for i in ls]

ValueError: invalid literal for int() with base 10: '1.0'

In [13]:
type(data['label'][0])

numpy.float64

In [21]:
data['filter'] = filter_

In [22]:
with open(os.path.join(os.getcwd(), '../Data/contents.csv'), 'w') as f:
    data[['label', 'filter']].to_csv(f)